# Fantasy Football Schedule Generator
In this tutorial I will show you how to:
1. Get the teams from yahoo
2. Generate the schedule with various options
3. Validate the schedule
4. Upload the schedule (manual for now but might make a vid on automating it)


## Get the Schedule

In [65]:
#Copy and paste the text for your league. 
team_str="""
logo	Ben’s Koo Young Hoes	Tom	Jul 6	Created League
logo	The Half Chubbs	Joshua	Jul 6	Joined via renew
logo	Bend It Like Beckham	Mariano	Jul 6	Joined via renew
logo	Football Team	Eric	Jul 6	Joined via renew
logo	Keep Dragon Shoulder	Greg	Jul 6	Joined via renew
logo	#3	Matt	Jul 6	Joined via renew
logo	Stafford Infection	Eric	Jul 6	Joined via renew
logo	Jake's Liver 2	Jake	Jul 6	Joined via renew
logo	AutodraftKings	Nick	Jul 6	Joined via renew
logo	White Sox Hat	Lucas	Jul 6	Joined via renew
logo	7th Floor Crew	Edwin	Jul 6	Joined via renew
logo	Real Edwards	Jason	Jul 6	Joined via renew
logo	Just Win Baby	Matteo	Jul 6	Joined via renew
logo	Bishop Sycamore	Chris	Jul 6	Joined via renew
logo	OnlyFants	Ankit	Jul 6	Joined via renew
logo	Mack Top Greg Bottom	Nick	Jul 6	Joined via renew
"""

In [66]:
all_teams=[x.split("	")[1] for x in team_str.split("\n")[1:-1]]
all_teams,len(all_teams)

(['Ben’s Koo Young Hoes',
  'The Half Chubbs',
  'Bend It Like Beckham',
  'Football Team',
  'Keep Dragon Shoulder',
  '#3',
  'Stafford Infection',
  "Jake's Liver 2",
  'AutodraftKings',
  'White Sox Hat',
  '7th Floor Crew',
  'Real Edwards',
  'Just Win Baby',
  'Bishop Sycamore',
  'OnlyFants',
  'Mack Top Greg Bottom'],
 16)

In [67]:
import random
random.shuffle(all_teams)#shuffles in place. Makes everything "random" -- technically pseudorandom but still should be good enough
all_teams

['Football Team',
 'Mack Top Greg Bottom',
 'The Half Chubbs',
 'Ben’s Koo Young Hoes',
 'Stafford Infection',
 'Just Win Baby',
 "Jake's Liver 2",
 'Bend It Like Beckham',
 'AutodraftKings',
 'White Sox Hat',
 '#3',
 'Real Edwards',
 '7th Floor Crew',
 'Keep Dragon Shoulder',
 'OnlyFants',
 'Bishop Sycamore']

To preserve the randomness you can set a "seed"  https://docs.python.org/3/library/random.html#random.seed but we just save the output later instead.

Since we randomized earlier we can use deterministic methods like %, the modulo operator, to simplify our code.

In [68]:
from collections import defaultdict#makes list append simpler

divisions=defaultdict(list)
for i,team in enumerate(all_teams):
    divisions[i//4].append(team)#every 4th team from the random list is assigned to the same division
divisions

defaultdict(list,
            {0: ['Football Team',
              'Mack Top Greg Bottom',
              'The Half Chubbs',
              'Ben’s Koo Young Hoes'],
             1: ['Stafford Infection',
              'Just Win Baby',
              "Jake's Liver 2",
              'Bend It Like Beckham'],
             2: ['AutodraftKings', 'White Sox Hat', '#3', 'Real Edwards'],
             3: ['7th Floor Crew',
              'Keep Dragon Shoulder',
              'OnlyFants',
              'Bishop Sycamore']})

In [69]:

conferences=defaultdict(list)
conference_nums=[]
for division, teams in divisions.items():
    division_num=len(teams)//2#2 conferences
    conferences[division//division_num]+=teams
    conference_nums.append(division_num)
    
conferences


defaultdict(list,
            {0: ['Football Team',
              'Mack Top Greg Bottom',
              'The Half Chubbs',
              'Ben’s Koo Young Hoes',
              'Stafford Infection',
              'Just Win Baby',
              "Jake's Liver 2",
              'Bend It Like Beckham'],
             1: ['AutodraftKings',
              'White Sox Hat',
              '#3',
              'Real Edwards',
              '7th Floor Crew',
              'Keep Dragon Shoulder',
              'OnlyFants',
              'Bishop Sycamore']})

In [70]:
game_types={
    "divisional": 4,#Mandate a divisional game to start and stop the season - hard coded below
    "conference": 4,
    "non_conference": 4
}
game_types_per_week=[]
for k,v in game_types.items():
    game_types_per_week+=[k]*v
random.shuffle(game_types_per_week)
game_types_per_week.insert(0,"divisional")
game_types_per_week+=["divisional"]
game_types_per_week

['divisional',
 'non_conference',
 'divisional',
 'non_conference',
 'conference',
 'divisional',
 'divisional',
 'conference',
 'conference',
 'conference',
 'non_conference',
 'divisional',
 'non_conference',
 'divisional']

# Constraints 
- 1 game max between nondivisional teams. 
- 2 games between divisional

These constraints are relatively simple. Each constraint can be coded by a corresponding universe for each team. To simplify things we will not mix weeks, each week has a shared theme: divisional, non_conference, and conference. When a game is played we will remove that opponent from each teams' universe. 
  
If you wanted to add a "don't play a team 2 weeks in a row rule" you would need to add a short term dont_play set for each team to include all the invalid teams for that team, for that week. 

In [71]:
from itertools import combinations,permutations
import copy


divisional_opponent_universe={}
conference_opponent_universe={}
nonconference_opponent_universe={}

for division, teams in divisions.items():
    for team in teams:
        divisional_opponent_universe[team]=list(set(teams)-set([team]))*2

for conference, teams in conferences.items():
    for team in teams:
        conference_opponent_universe[team]=list(set(teams)-set([team])-set(divisional_opponent_universe[team]))
        #copy is helpful because of pointer vs primitive rules in Python. It may not be necessary in this case though
        nonconference_opponent_universe[team]=copy.deepcopy(conferences[(conference+1)%2])#with 2 conferences this maps 0 to 1 and 1 to 0

universes={
    "divisional": divisional_opponent_universe,
    "conference": conference_opponent_universe,
    "non_conference": nonconference_opponent_universe,
}

universes['divisional']

{'Football Team': ['The Half Chubbs',
  'Ben’s Koo Young Hoes',
  'Mack Top Greg Bottom',
  'The Half Chubbs',
  'Ben’s Koo Young Hoes',
  'Mack Top Greg Bottom'],
 'Mack Top Greg Bottom': ['The Half Chubbs',
  'Football Team',
  'Ben’s Koo Young Hoes',
  'The Half Chubbs',
  'Football Team',
  'Ben’s Koo Young Hoes'],
 'The Half Chubbs': ['Football Team',
  'Ben’s Koo Young Hoes',
  'Mack Top Greg Bottom',
  'Football Team',
  'Ben’s Koo Young Hoes',
  'Mack Top Greg Bottom'],
 'Ben’s Koo Young Hoes': ['The Half Chubbs',
  'Football Team',
  'Mack Top Greg Bottom',
  'The Half Chubbs',
  'Football Team',
  'Mack Top Greg Bottom'],
 'Stafford Infection': ["Jake's Liver 2",
  'Just Win Baby',
  'Bend It Like Beckham',
  "Jake's Liver 2",
  'Just Win Baby',
  'Bend It Like Beckham'],
 'Just Win Baby': ['Stafford Infection',
  'Bend It Like Beckham',
  "Jake's Liver 2",
  'Stafford Infection',
  'Bend It Like Beckham',
  "Jake's Liver 2"],
 "Jake's Liver 2": ['Stafford Infection',
  'Just

Notice how divisional has each time in there twice. The other universes are self explanatory

In [72]:

def gen_matchups(all_teams_, universe_):
    '''
    Attempts to generate a week of matchups using each teams universe ()
    '''
    universe_=copy.deepcopy(universe_)
    weekly_teams_playing = set()
    weekly_matchups = []
    teams_remaining=set(all_teams_)
    
    while teams_remaining:
        team_1=teams_remaining.pop()
        candidates=list(set(universe_[team_1])-weekly_teams_playing)
        team_2=random.choice(candidates)
        universe_[team_1].remove(team_2)
        universe_[team_2].remove(team_1)

        weekly_teams_playing=weekly_teams_playing.union(set([team_1,team_2]))
        weekly_matchups.append([team_1,team_2])
        teams_remaining.remove(team_2)
    return weekly_matchups

In [73]:
weekly_teams_playing=defaultdict(set)
games_played={}
weekly_matchups=[]




for week, game_type in enumerate(game_types_per_week):
    #print(week)
        
    print(week,game_type)
    matchups=None
    #Potentially invalid matchups can happen depending on the contstraints so each week will have a given number of attempts. 
    #It's also possible a prior week leads to an impossible state. We will only support rerunning on the week level
    ATTEMPTS=100
    for attempt in range(ATTEMPTS):
        try:
            matchups=gen_matchups(all_teams,universes[game_type])
            break
        except:
            continue
    if not matchups:
        raise 
    else:
        for name, universe in universes.items():
            for team_1,team_2 in matchups:
                for a,b in permutations([team_1,team_2], 2):
                    if b in universe.get(a,[]):                        
                        universes[name][a].remove(b)

        weekly_matchups.append(matchups)

[x[0] for x in weekly_matchups]

0 divisional
1 non_conference
2 divisional
3 non_conference
4 conference
5 divisional
6 divisional
7 conference
8 conference
9 conference
10 non_conference
11 divisional
12 non_conference
13 divisional


[['7th Floor Crew', 'OnlyFants'],
 ['7th Floor Crew', "Jake's Liver 2"],
 ['7th Floor Crew', 'Keep Dragon Shoulder'],
 ['7th Floor Crew', 'Football Team'],
 ['7th Floor Crew', 'White Sox Hat'],
 ['7th Floor Crew', 'Keep Dragon Shoulder'],
 ['7th Floor Crew', 'Bishop Sycamore'],
 ['7th Floor Crew', 'AutodraftKings'],
 ['7th Floor Crew', '#3'],
 ['7th Floor Crew', 'Real Edwards'],
 ['7th Floor Crew', 'Bend It Like Beckham'],
 ['7th Floor Crew', 'OnlyFants'],
 ['7th Floor Crew', 'The Half Chubbs'],
 ['7th Floor Crew', 'Bishop Sycamore']]

# Validating the Output
We will output the occurrences of each matchup.  
  
No row should have a value greater than 2. And we should see nice c

In [74]:
import numpy as np
N=len(all_teams)
occurences_matrix=np.zeros((N,N))

for weeks_matchups in weekly_matchups:
    
    for team_1,team_2 in weeks_matchups:
        for a,b in permutations([team_1,team_2], 2):
            occurences_matrix[all_teams.index(a)][all_teams.index(b)]+=1
occurences_matrix

array([[0., 2., 2., 2., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0.],
       [2., 0., 2., 2., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0.],
       [2., 2., 0., 2., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0.],
       [2., 2., 2., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1.],
       [1., 1., 1., 1., 0., 2., 2., 2., 0., 1., 1., 1., 0., 0., 0., 1.],
       [1., 1., 1., 1., 2., 0., 2., 2., 1., 1., 1., 0., 0., 1., 0., 0.],
       [1., 1., 1., 1., 2., 2., 0., 2., 0., 0., 0., 1., 1., 0., 1., 1.],
       [1., 1., 1., 1., 2., 2., 2., 0., 1., 0., 0., 0., 1., 0., 1., 1.],
       [1., 0., 0., 1., 0., 1., 0., 1., 0., 2., 2., 2., 1., 1., 1., 1.],
       [0., 1., 1., 0., 1., 1., 0., 0., 2., 0., 2., 2., 1., 1., 1., 1.],
       [1., 1., 0., 0., 1., 1., 0., 0., 2., 2., 0., 2., 1., 1., 1., 1.],
       [1., 0., 1., 0., 1., 0., 1., 0., 2., 2., 2., 0., 1., 1., 1., 1.],
       [1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 2., 2., 2.],
       [0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1.,

# Input into Yahoo

In [75]:

import pandas as pd

matchups_df=pd.DataFrame(weekly_matchups).T
matchups_df.columns=[x+1 for x in matchups_df.columns]
matchups_df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,"[7th Floor Crew, OnlyFants]","[7th Floor Crew, Jake's Liver 2]","[7th Floor Crew, Keep Dragon Shoulder]","[7th Floor Crew, Football Team]","[7th Floor Crew, White Sox Hat]","[7th Floor Crew, Keep Dragon Shoulder]","[7th Floor Crew, Bishop Sycamore]","[7th Floor Crew, AutodraftKings]","[7th Floor Crew, #3]","[7th Floor Crew, Real Edwards]","[7th Floor Crew, Bend It Like Beckham]","[7th Floor Crew, OnlyFants]","[7th Floor Crew, The Half Chubbs]","[7th Floor Crew, Bishop Sycamore]"
1,"[Ben’s Koo Young Hoes, The Half Chubbs]","[Ben’s Koo Young Hoes, Bishop Sycamore]","[Ben’s Koo Young Hoes, Football Team]","[Ben’s Koo Young Hoes, Keep Dragon Shoulder]","[Ben’s Koo Young Hoes, Jake's Liver 2]","[Ben’s Koo Young Hoes, The Half Chubbs]","[Ben’s Koo Young Hoes, Mack Top Greg Bottom]","[Ben’s Koo Young Hoes, Stafford Infection]","[Ben’s Koo Young Hoes, Just Win Baby]","[Ben’s Koo Young Hoes, Bend It Like Beckham]","[Ben’s Koo Young Hoes, AutodraftKings]","[Ben’s Koo Young Hoes, Mack Top Greg Bottom]","[Ben’s Koo Young Hoes, OnlyFants]","[Ben’s Koo Young Hoes, Football Team]"
2,"[Just Win Baby, Bend It Like Beckham]","[Just Win Baby, AutodraftKings]","[Just Win Baby, Jake's Liver 2]","[Just Win Baby, White Sox Hat]","[Just Win Baby, The Half Chubbs]","[Just Win Baby, Jake's Liver 2]","[Just Win Baby, Stafford Infection]","[Just Win Baby, Mack Top Greg Bottom]","[Mack Top Greg Bottom, Stafford Infection]","[Just Win Baby, Football Team]","[Just Win Baby, #3]","[Just Win Baby, Stafford Infection]","[Just Win Baby, Keep Dragon Shoulder]","[Just Win Baby, Bend It Like Beckham]"
3,"[Mack Top Greg Bottom, Football Team]","[Mack Top Greg Bottom, Keep Dragon Shoulder]","[Mack Top Greg Bottom, The Half Chubbs]","[Mack Top Greg Bottom, #3]","[Mack Top Greg Bottom, Bend It Like Beckham]","[Mack Top Greg Bottom, Football Team]","[Bend It Like Beckham, Jake's Liver 2]","[Bend It Like Beckham, The Half Chubbs]","[Bend It Like Beckham, Football Team]","[Mack Top Greg Bottom, Jake's Liver 2]","[Mack Top Greg Bottom, OnlyFants]","[Bend It Like Beckham, Jake's Liver 2]","[Mack Top Greg Bottom, White Sox Hat]","[Mack Top Greg Bottom, The Half Chubbs]"
4,"[Real Edwards, AutodraftKings]","[Bend It Like Beckham, OnlyFants]","[Bend It Like Beckham, Stafford Infection]","[Bend It Like Beckham, AutodraftKings]","[Real Edwards, Keep Dragon Shoulder]","[Bend It Like Beckham, Stafford Infection]","[Real Edwards, #3]","[Real Edwards, OnlyFants]","[Real Edwards, Bishop Sycamore]","[Bishop Sycamore, #3]","[Real Edwards, Football Team]","[Real Edwards, White Sox Hat]","[Bend It Like Beckham, Bishop Sycamore]","[Real Edwards, White Sox Hat]"
5,"[Bishop Sycamore, Keep Dragon Shoulder]","[Real Edwards, Stafford Infection]","[Real Edwards, AutodraftKings]","[Real Edwards, The Half Chubbs]","[Bishop Sycamore, AutodraftKings]","[Real Edwards, #3]","[The Half Chubbs, Football Team]","[Bishop Sycamore, White Sox Hat]","[The Half Chubbs, Jake's Liver 2]","[The Half Chubbs, Stafford Infection]","[Bishop Sycamore, Jake's Liver 2]","[Bishop Sycamore, Keep Dragon Shoulder]","[Real Edwards, Jake's Liver 2]","[AutodraftKings, #3]"
6,"[#3, White Sox Hat]","[The Half Chubbs, White Sox Hat]","[Bishop Sycamore, OnlyFants]","[Bishop Sycamore, Stafford Infection]","[#3, OnlyFants]","[Bishop Sycamore, OnlyFants]","[AutodraftKings, White Sox Hat]","[#3, Keep Dragon Shoulder]","[AutodraftKings, Keep Dragon Shoulder]","[AutodraftKings, OnlyFants]","[The Half Chubbs, Keep Dragon Shoulder]","[The Half Chubbs, Football Team]","[AutodraftKings, Football Team]","[Keep Dragon Shoulder, OnlyFants]"
7,"[Jake's Liver 2, Stafford Infection]","[#3, Football Team]","[#3, White Sox Hat]","[Jake's Liver 2, OnlyFants]","[Football Team, Stafford Infection]","[AutodraftKings, White Sox Hat]","[Keep Dragon Shoulder, OnlyFants]","[Jake's Liver 2, Football Team]","[White Sox Hat, OnlyFants]","[Keep Dragon Shoulder, White Sox Hat]","[White Sox Hat, Stafford Infection]","[Autod

In [76]:
import pandas as pd
pd.DataFrame(weekly_matchups).to_csv("matchups.csv")#save for posterity

In [77]:
divisions

defaultdict(list,
            {0: ['Football Team',
              'Mack Top Greg Bottom',
              'The Half Chubbs',
              'Ben’s Koo Young Hoes'],
             1: ['Stafford Infection',
              'Just Win Baby',
              "Jake's Liver 2",
              'Bend It Like Beckham'],
             2: ['AutodraftKings', 'White Sox Hat', '#3', 'Real Edwards'],
             3: ['7th Floor Crew',
              'Keep Dragon Shoulder',
              'OnlyFants',
              'Bishop Sycamore']})

In [78]:
WEEK_NUM=1
print(game_types_per_week[WEEK_NUM-1])
weekly_matchups[WEEK_NUM-1]

divisional


[['7th Floor Crew', 'OnlyFants'],
 ['Ben’s Koo Young Hoes', 'The Half Chubbs'],
 ['Just Win Baby', 'Bend It Like Beckham'],
 ['Mack Top Greg Bottom', 'Football Team'],
 ['Real Edwards', 'AutodraftKings'],
 ['Bishop Sycamore', 'Keep Dragon Shoulder'],
 ['#3', 'White Sox Hat'],
 ["Jake's Liver 2", 'Stafford Infection']]

# Additional Information
Optimization theory and constraint programming deal with scheduling problem. It can be very, very complex. 
- [How the NFL may deal with it](https://www.math.cmu.edu/~af1p/Teaching/OR2/Projects/P56/OR-Final-Paper.pdf)
- [Constraint Programming wikipedia page](https://en.wikipedia.org/wiki/Constraint_programming#:~:text=Constraint%20programming%20(CP)%20is%20a,a%20set%20of%20decision%20variables.)
- [Coursera Course](https://www.coursera.org/learn/discrete-optimization) -- I took this, kinda hard I think I flunked by the end but did learn some useful stuff.